## 네이버 영화 리뷰 감성분석
- 네이버 영화 리뷰 감성 분류하기(https://wikidocs.net/44249) 참조

In [1]:
import pandas as pd
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [2]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


#### 1. 데이터 전처리

- train data

In [4]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [5]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [6]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [7]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [8]:
# 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- test data

In [9]:
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [10]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

#### 2. 텍스트 전처리

- train data

In [12]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [13]:
# 한글이 없는 글 --> ''만 남게됨
# ''만 남은 데이터는 제거: np.nan으로 대체후 dropna 실행
import numpy as np
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [14]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test data

In [15]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

In [16]:
train_df.to_csv('data/네이버영화리뷰 train 전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('data/네이버영화리뷰 test 전처리완료.tsv', sep='\t', index=False)

#### 3. 한글 처리

In [17]:
from konlpy.tag import Okt
okt = Okt()

In [18]:
text = '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [19]:
okt.morphs(text, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [21]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
print(lines[0])

이	VCP	0.018279601



In [22]:
stop_words = []
for line in lines:
    word = line.split('\t')[0]
    stop_words.append(word)

In [24]:
stop_words = [line.split('\t')[0] for line in lines]
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [25]:
# 형태소 분석기를 돌려서 나온 결과에서 불용어 제거하기
morphs = okt.morphs(text, stem=True)
clean_morphs = [morph for morph in morphs if morph not in stop_words]
clean_morphs

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [26]:
# Feature 변환을 위한 Vectorizer의 입력은 리스트가 아닌 문자열이어야 함
clean_text = ' '.join(clean_morphs)
clean_text

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

- train/test dataset에 적용

In [27]:
from tqdm import tqdm

X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_train.append(clean_morph_review)

100%|██████████| 145393/145393 [08:39<00:00, 279.80it/s]


In [28]:
%%time
X_test = []
for review in test_df.document:
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_test.append(clean_morph_review)

CPU times: total: 3min 15s
Wall time: 3min 9s


In [29]:
y_train = train_df.label.values
y_test = test_df.label.values

In [30]:
df = pd.DataFrame({'X': X_train, 'y': y_train})
df.to_csv('data/네이버영화리뷰 train 형태소처리완료.tsv', index=False)

In [31]:
df = pd.DataFrame({'X': X_test, 'y': y_test})
df.to_csv('data/네이버영화리뷰 test 형태소처리완료.tsv', index=False)

#### 4. 피쳐 변환 + 모델 학습/평가

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [33]:
pipeline = Pipeline([
    ('CVECT', CountVectorizer()), ('NB', MultinomialNB())
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('CVECT', CountVectorizer()), ('NB', MultinomialNB())])

In [34]:
pipeline.score(X_test, y_test)

0.827581265864243

#### 5. 실제 데이터 테스트

In [35]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ', '이 영화 핵노잼 ㅠㅠ', '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?', '와 개쩐다 정말 세계관 최강자들의 영화다'
]

In [36]:
# 전처리 - 한글 이외의 문자는 공백으로 변환
import re
reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [37]:
# 형태소 분석 후 불용어 제거
processed_reviews = []
for review in reviews:
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    processed_reviews.append(clean_morph_review)

In [38]:
# 예측
pipeline.predict(processed_reviews)

array([1, 0, 0, 0, 1], dtype=int64)

In [39]:
pipeline.predict_proba(processed_reviews)

array([[0.06260368, 0.93739632],
       [0.91612638, 0.08387362],
       [0.9906787 , 0.0093213 ],
       [0.9297786 , 0.0702214 ],
       [0.09745728, 0.90254272]])

#### 6. 최적 파라메터 찾기

In [40]:
from sklearn.model_selection import GridSearchCV
params = {'CVECT__ngram_range':[(1,1),(1,2)], 'CVECT__max_df':[0.95,0.98]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 59.1 s
Wall time: 53 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('NB', MultinomialNB())]),
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [41]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2)}

In [42]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.843936788667813